Corrección de cadenas de texto
===

* Ultima actualización: Marzo 6, 2023 | [YouTube](https://www.youtube.com/watch?v=4DzusT3Nl-Q&list=PLEFpZ3YehTnDX6z1fx4rT0wkt80ythBhc&index=12)

In [2]:
!pip3 install --quiet fuzzywuzzy
!pip3 install --quiet fuzzywuzzy python-Levenshtein

In [3]:
import pandas as pd

pd.__version__

'1.5.3'

Comparación de cadenas de texto
---

In [4]:
from fuzzywuzzy import fuzz, process

#
# Comparación de strings
#
display(
    fuzz.WRatio("hola mundo", "hola mundo"),
    fuzz.WRatio("hola mundo", "Hola mundo"),
    fuzz.WRatio("hola mundo", "Hola Mundo"),
    fuzz.WRatio("hola mundo", "hola mundo!"),
)

100

100

100

100

In [5]:
#
# Comparación de strings
#
display(
    fuzz.WRatio("hola mundo!", "hola mundo"),
    fuzz.WRatio("hola mundo", "Hola mundo!"),
    fuzz.WRatio("hola mundo", "Hola mundoo"),
    fuzz.WRatio("hola mundo", "hola mundooo"),
    fuzz.WRatio("hola mundo", "hola mundoooo"),
    fuzz.WRatio("hola mundo", "hola"),
    fuzz.WRatio("hola", "hola mundo"),
)

100

100

95

91

87

90

90

In [6]:
#
# Comparación con arrays
#
base_string = "hola mundo"
array = pd.Series(
    [
        "Hola mundo",
        "mundo",
        "Hola",
        "Hola mundo cruel!",
    ]
)


process.extract(
    base_string,
    array,
)

[('Hola mundo', 100, 0),
 ('mundo', 90, 1),
 ('Hola', 90, 2),
 ('Hola mundo cruel!', 90, 3)]

Corrección y unificación básica cuando se conoce la clave
----

In [7]:
%%writefile /tmp/data.csv
ciudad
Medellin
Bogota
bogota
bogote
Mdellin
medellin
Cali
cali
CALI

Writing /tmp/data.csv


In [8]:
df = pd.read_csv("/tmp/data.csv")
df

,ciudad
0,Medellin
1,Bogota
2,bogota
3,bogote
4,Mdellin
5,medellin
6,Cali
7,cali
8,CALI


El parámetro "`limit`" en la función `process.extract()` de la librería **FuzzyWuzzy** en Python define el número máximo de opciones que se deben devolver en la lista de resultados. Por defecto, este valor es 5, lo que significa que solo se devolverán las 5 mejores opciones clasificadas por su puntaje de similitud. Sin embargo, si se especifica un valor diferente para "`limit`", la función devolverá un número máximo de opciones igual a ese valor.

Por ejemplo, si se llama a la función `process.extract()` con `limit=3`, la función devolverá una lista de hasta 3 opciones clasificadas por su puntaje de similitud en lugar de las 5 opciones predeterminadas.

In [9]:
#
# Nombres validos
#
valid_names = ["Medellin", "Bogota", "Cali"]
df["ciudad_"] = df.ciudad.copy()

#
# Valor mínimo de similitud para hacer el cambio
#
min_threshold = 80

#
# Estructuta básica
#
for valid_name in valid_names:

    potential_matches = process.extract(
        valid_name,
        df.ciudad,
        limit=df.shape[0],
    )

    for potential_match in potential_matches:

        if potential_match[1] >= min_threshold:

            df.loc[df.ciudad == potential_match[0], "ciudad_"] = valid_name

df

,ciudad,ciudad_
0,Medellin,Medellin
1,Bogota,Bogota
2,bogota,Bogota
3,bogote,Bogota
4,Mdellin,Medellin
5,medellin,Medellin
6,Cali,Cali
7,cali,Cali
8,CALI,Cali
